In [3]:
from hidromet import merge
from datetime import date
from hidromet import config
from hidromet import grade
import xarray as xr
import pandas as pd
import geopandas as gpd

### Obter dados de satélite para um período

Os dados de satélite serão utilizado para o preenchimento de falhas das séries e das áreas que não possuem representatividade pelo buffer padrão utilizado para a área de influência dos postos, disponível no arquivo de configurações `config.py` do projeto. 

> OBS1: O download da série completa só é necessária para o preenchimento do histórico. Operacionalmente, os dados diários de satélite podem ser baixados utilizando apenas a data atual.

> OBS2: Após o download, os arquivos grib são convertidos para netcdf para facilitar a manipulação.

**ATENÇÃO** Por conta da coleta e extração de dados, a execução deste notebook pode demorar cerca de algumas horas.

In [4]:
arquivos_baixados = list(config.dir_merge.glob("*.nc"))

#### Setar período de download

In [5]:
periodo = pd.date_range(start="20000602", end=date.today(), freq="D")

#### Baixar dados de um período

In [6]:
for data in periodo:
    data_fmt = data.strftime("%Y%m%d")
    nome_arquivo = f"MERGE_CPTEC_{data_fmt}.nc"
    arquivo_merge = config.dir_merge / nome_arquivo
    
    if arquivo_merge not in arquivos_baixados:
        try:
            arquivo_grib = merge.obter(data=data)
            nome_arquivo = arquivo_grib.stem
            arquivo_netcdf = config.dir_merge.joinpath(f"{nome_arquivo}.nc")
            grade.converter_grib_para_netcdf(arquivo_grib=arquivo_grib, arquivo_netcdf=arquivo_netcdf)
        except (FileNotFoundError):
            print(f"Arquivo {arquivo_grib} não encontrado.")
            continue
    else:
        print(f"O arquivo {nome_arquivo} já foi baixado")


O arquivo MERGE_CPTEC_20000602.nc já foi baixado
O arquivo MERGE_CPTEC_20000603.nc já foi baixado
O arquivo MERGE_CPTEC_20000604.nc já foi baixado
O arquivo MERGE_CPTEC_20000605.nc já foi baixado
O arquivo MERGE_CPTEC_20000606.nc já foi baixado
O arquivo MERGE_CPTEC_20000607.nc já foi baixado
O arquivo MERGE_CPTEC_20000608.nc já foi baixado
O arquivo MERGE_CPTEC_20000609.nc já foi baixado
O arquivo MERGE_CPTEC_20000610.nc já foi baixado
O arquivo MERGE_CPTEC_20000611.nc já foi baixado
O arquivo MERGE_CPTEC_20000612.nc já foi baixado
O arquivo MERGE_CPTEC_20000613.nc já foi baixado
O arquivo MERGE_CPTEC_20000614.nc já foi baixado
O arquivo MERGE_CPTEC_20000615.nc já foi baixado
O arquivo MERGE_CPTEC_20000616.nc já foi baixado
O arquivo MERGE_CPTEC_20000617.nc já foi baixado
O arquivo MERGE_CPTEC_20000618.nc já foi baixado
O arquivo MERGE_CPTEC_20000619.nc já foi baixado
O arquivo MERGE_CPTEC_20000620.nc já foi baixado
O arquivo MERGE_CPTEC_20000621.nc já foi baixado
O arquivo MERGE_CPTE

#### Obter dados do dia atual

**ATENÇÃO** Sujeito à disponibilidade do FTP. Consultar site `http://ftp.cptec.inpe.br/modelos/tempo/MERGE/GPM/DAILY/` para informações sobre os dados disponíveis.

In [7]:
hoje = date.today()

In [8]:
data_fmt = hoje.strftime("%Y%m%d")
nome_arquivo = f"MERGE_CPTEC_{data_fmt}.nc"
arquivo_merge = config.dir_merge / nome_arquivo

if arquivo_merge not in arquivos_baixados:
    try:
        arquivo_grib = merge.obter(data=hoje)
        nome_arquivo = arquivo_grib.stem
        arquivo_netcdf = config.dir_merge.joinpath(f"{nome_arquivo}.nc")
        grade.converter_grib_para_netcdf(arquivo_grib=arquivo_grib, arquivo_netcdf=arquivo_netcdf)
    except (FileNotFoundError):
        print(f"Arquivo {arquivo_grib} não encontrado")   
else:
    print(f"O arquivo {nome_arquivo} já foi baixado")

O arquivo MERGE_CPTEC_20211107.nc já foi baixado


### Mesclar dados de satélite obtidos

In [9]:
arquivos_netcdf = list(config.dir_merge.glob("*.nc"))
ds = xr.open_mfdataset(arquivos_netcdf, combine='nested', concat_dim='valid_time').sortby('valid_time')

### Recortar dataset com o contorno da bacia

#### Preparar contorno da bacia

In [10]:
gdf = gpd.read_file(config.dir_contorno)
gdf.set_crs(epsg=config.epsg_inicial, inplace=True)
bacia = gdf.iloc[0]
nome_bacia = bacia.bacia

#### Recortar o dataset

In [11]:
_ds = grade.preparar_para_recorte(ds)
ds_recortado = grade.recorte(_ds, bacia)
ds_limpo = ds_recortado.drop(["time", "step", "surface", "prmsl"])
# extraindo chunks do dataset
ds_valido = ds_limpo.load()

#### Exportar dataset recortado

In [12]:
dir_netcdf_concatenado = config.dir_merge_concat.joinpath("merge_concatenado.nc")
ds_valido.to_netcdf(dir_netcdf_concatenado)

In [13]:
dir_netcdf_concatenado

PosixPath('/mnt/coworking/landrade/datathons-2/arquivos/merge-concatenado/merge_concatenado.nc')